# Pythonによる並行処理のパターン
+ https://qiita.com/kaitolucifer/items/e4ace07bd8e112388c75

#### Pythonでマルチタスクを同時に処理したい時は主に2通りのやり方
+ プライマリースレッドを持つプロセスを複数個立ち上げて, 複数のタスクを処理する
+ 1個のプロセスの中で複数スレッドを立ち上げて, 複数のタスクを処理する
+ 複数プロセスで複数スレッドを立ち上げて複数のタスクを処理もできるがモデルが複雑化するので非推奨

#### Pythonのスレッド
+ プロセスでシミュレートしたような疑似スレッドではなくOSのネイティブスレッドを使用する
+ Linux系ではPOSIXスレッド
+ WindowsではWindowsスレッド

2つのモジュールが用意されている  
+ _thread 低レベルモジュール
+ threading 高レベルモジュール

In [1]:
from type_hint import *
from import_str import importstr
from log_conf import logging
log = logging.getLogger('nb')

# log.setLevel(logging.WARN)
log.setLevel(logging.INFO)
log.setLevel(logging.DEBUG)

module_parent_dir /Users/inoueshinichi/Desktop/MyGithub/Learn_PythonConcurrent/..
module_parent_dir /Users/inoueshinichi/Desktop/MyGithub/Learn_PythonConcurrent/..
module_parent_dir /Users/inoueshinichi/Desktop/MyGithub/Learn_PythonConcurrent/..


ConcurrentPatternフォルダの各アプリクラスを実行する関数

In [2]:
# running everything app
def run(app, *argv):
    argv = list(argv)
    log.info('Running: {}({!r}).main()'.format(app, argv))
    print("*app.rsplit('.', 1) : ", *app.rsplit('.', 1))

    app_cls = importstr(*app.rsplit('.', 1)) # __import__を実行
    app_cls(argv).main()

    log.info("Finished: {}.({!r}).main()".format(app, argv))

#### 1-1. インスタンス化
t1とt2が交替で実行されていることが確認できます。交替ルールの1つはIO操作（ここではprint操作が該当する）

In [3]:
# run("concurrent_pattern.thread_app.SimpleThreadApp")

#### 1-2. カスタイマイズ
threading.Threadクラスを継承してrun()メソッドをオーバーライドして独自のカスタムスレッドを使用する

In [4]:
# run('concurrent_pattern.thread_app.ClassStyleThreadApp')

#### 1-3. スレッド数を計算
`active_count`でアクティブなスレッド数を数えることができる.

In [5]:
# run('concurrent_pattern.thread_app.CountingThreadApp')

#### 1-4. デーモンスレッド
+ デーモンスレッドという実態は存在しない. 
+ 通常のプライマリースレッドに対するワーカースレッドのことをデーモンスレッドと呼んでいるだけ
+ デーモンスレッドはプライマリースレッドの終了まで破棄が遅延される

In [6]:
# run('concurrent_pattern.thread_app.DaemonThreadApp')

![GIL.jpeg](image/GIL.jpg)

![CPythonThreading](image/CPythonThread.jpg)

![BoundTask](image/BoundTask_ConvoyEffect.jpg)

#### データ競合(Data Race)

![分解不可操作(Atomic)](image/DecompAtomicOpe.jpg)

![DataRace](image/DataRace.jpg)

In [7]:
# run('concurrent_pattern.data_race.DataRaceThreadApp')

#### 1-6-1. 排他制御(mutex)

In [8]:
# run('concurrent_pattern.mutex_for_datarace.MutexForDataRaceApp')

#### 1-6-2. 再帰排他制御(recursive_mutex)

In [9]:
# run('concurrent_pattern.mutex_for_datarace.RecursiveMutexForDataRaceApp')

#### 1-6-3. 有限セマフォ（BoundedSemaphore）制御

+ 排他制御は、ある時刻において、リソースを処理できるのは1つのスレッドのみに制限する
+ セマフォは一定数のスレッドの同時処理を許容する制限
+ トイレに3つの便座があって、同時に3人が使っていて他の人は並んで待つシチュエーションがセマフォに該当

In [10]:
# run('concurrent_pattern.semaphore.BoundedSemaphoreApp')

#### 1-6-4. イベント（Event）制御

スレッドのイベントはメインスレッドが他のスレッドをコントロールするためのもの  

| メソッド | 説明 |
| :-- | :-- |
| clear | flagをFalseにする |
| set | flagをTrueにする |
| is_set | flagがTrueのときTrueを返す |
| wait | flagをモニタリングし続ける. flagがFalseの時はブロッキング（blocking）する |


In [11]:
# run('concurrent_pattern.event.EventApp')

#### 1-6-5. タイマー（Timer）制御

タイマーでスレッドを起動できる

In [12]:
run('concurrent_pattern.timer.TimerThreadApp')

2023-06-27 17:20:48,370 INFO     pid:11506 nb:004:run Running: concurrent_pattern.timer.TimerThreadApp([]).main()


*app.rsplit('.', 1) :  concurrent_pattern.timer TimerThreadApp
module_parent_dir /Users/inoueshinichi/Desktop/MyGithub/Learn_PythonConcurrent/concurrent_pattern/..


2023-06-27 17:20:48,548 INFO     pid:11506 concurrent_pattern.timer:054:main Starting TimerThreadApp, Namespace()
2023-06-27 17:20:49,555 INFO     pid:11506 nb:010:run Finished: concurrent_pattern.timer.TimerThreadApp.([]).main()


Start thread by timer!
[primary-thrad] Shutdown...


#### 1-6-6. 条件（Condition）制御

+ 条件判定でスレッドを制御する
+ Conditionクラス

| メソッド | 説明 |
| :-- | :-- |
| wait | 通知されるか引数のtimeout時間に達するまでスレッドをハングアップする |
| notify | ハングアップされたスレッド（デフォルトn=1）に通知する. ロックを取得した状態でしか使えない |
| notifyAll | ハングアップされた全てのスレッドに通知する |
